In [9]:
# ------ Case Folding ---------

import pandas as pd
import numpy as np


data = pd.read_json("dataset.json", lines=True)
data.head()
data['text'] = data['text'].str.lower()
print('Case Folding Result : \n')
print(data['text'].head(5))
print('\n\n\n')

Case Folding Result : 

0    #darkhumor #darkjokes #darkjoke @darkhumour247...
1    desain rmah buat pecinta darkjokes https://t.c...
2    @detikcom onani 10 tahun? mending korupsi dong...
3    @woochy_ @ondongmin @ickfryo @preciouschamsae ...
4    pembahasan politik dan pemerintahan di kelas s...
Name: text, dtype: object






In [10]:
# ------ Tokenizing & Cleansing ---------
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist


def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
data['text'] = data['text'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data['text'] = data['text'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data['text'] = data['text'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

data['text'] = data['text'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data['text'] = data['text'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

data['text'] = data['text'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data['tweet_tokens'] = data['text'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(data['tweet_tokens'].head(7))
print('\n\n\n')


Tokenizing Result : 

0                                              [follo]
1             [desain, rmah, buat, pecinta, darkjokes]
2    [onani, tahun, mending, korupsi, dong, cuma, t...
3    [sayang, jgn, marah, ayo, kita, natalan, hahah...
4    [pembahasan, politik, dan, pemerintahan, di, k...
5                                          [darkjokes]
6    [dirs, ini, baru, cuma, becanda, gak, usah, sa...
Name: tweet_tokens, dtype: object






In [8]:
# ------ Filtering / Stopword removing ---------

from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 
                       'amp', 'biar',  
                       'nih', 'sih', 
                       'si', 'tuh', 
                       'aja', 'n', 't', 
                       'nyg', 'pen', 'u', 'nan', 'loh', 'rt',
                       'yah'])


# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data['tweet_tokens_WSW'] = data['tweet_tokens'].apply(stopwords_removal) 


print(data['tweet_tokens_WSW'].head())

0                                              [follo]
1                   [desain, rmah, pecinta, darkjokes]
2                            [onani, mending, korupsi]
3    [sayang, jgn, marah, ayo, natalan, hahahhahaha...
4    [pembahasan, politik, pemerintahan, kelas, epi...
Name: tweet_tokens_WSW, dtype: object


In [7]:
# ------ Normalisasi ---------
normalizad_word = pd.read_excel("normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

data['tweet_normalized'] = data['tweet_tokens_WSW'].apply(normalized_term)

data['tweet_normalized'].head(10)

0                                              [follo]
1                  [desain, rumah, pecinta, darkjokes]
2                            [onani, mending, korupsi]
3    [sayang, jgn, marah, ayo, natalan, lucu, istig...
4    [pembahasan, politik, pemerintahan, kelas, epi...
5                                          [darkjokes]
6                         [dirs, bercanda, gak, sabar]
7    [kids, better, watch, out, santa, likes, no, w...
8    [think, allah, is, the, one, true, god, mean, ...
9    [spit, on, blind, children, and, tell, them, i...
Name: tweet_normalized, dtype: object

In [5]:
# ------ Stemming ---------

# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data['tweet_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data['tweet_tokens_stemmed'] = data['tweet_normalized'].swifter.apply(get_stemmed_term)
print(data['tweet_tokens_stemmed'])

16660
------------------------
follo : follo
desain : desain
rumah : rumah
pecinta : cinta
darkjokes : darkjokes
onani : onani
mending : mending
korupsi : korupsi
sayang : sayang
jgn : jgn
marah : marah
ayo : ayo
natalan : natal
lucu : lucu
istighfar : istighfar
takut : takut
dilanjut : lanjut
pembahasan : bahas
politik : politik
pemerintahan : perintah
kelas : kelas
epic : epic
gak : gak
bahas : bahas
bareng : bareng
dosen : dosen
senior : senior
dirs : dirs
bercanda : canda
sabar : sabar
kids : kids
better : better
watch : watch
out : out
santa : santa
likes : likes
no : no
witnesses : witnesses
think : think
allah : allah
is : is
the : the
one : one
true : true
god : god
mean : mean
universe : universe
did : did
start : start
with : with
big : big
explosion : explosion
spit : spit
on : on
blind : blind
children : children
and : and
tell : tell
them : them
its : its
raining : raining
bro : bro
this : this
paralyzed : paralyzed
kid : kid
at : at
my : my
school : school
so : so
funny :

ngatain : ngatain
fisik : fisik
dikit : dikit
dijadikan : jadi
konten : konten
himbauan : himbauan
pejuang keadilan sosial : juang adil sosial
nek : nek
nonton : nonton
beginian : begini
jek : jek
ra : ra
sah : sah
menghakimi : hakim
wkwk : wkwk
dibalik : balik
kebitteranku : kebitteranku
kulempar : lempar
liat : liat
pasangan : pasang
kakeknenek : kakeknenek
jalan : jalan
sasaranya : sasaranya
adalah : adalah
amerika : amerika
sebutan : sebut
popular : popular
kaya : kaya
gada : gada
batasan : batas
wajar : wajar
humour : humour
hadi : had
gari : gari
ikatae : ikatae
kwenda : kwenda
libur : libur
dibilang : bilang
divisi : divisi
kudu : kudu
ningkatin : ningkatin
prokes : prokes
minimalisir : minimalisir
kehad : had
syp : syp
ngajarin : ngajarin
papa : papa
yawlaaa : yawlaaa
woor : woor
see : see
benar : benar
bocah : bocah
mendalami : dalam
sibuk : sibuk
ngurusin : ngurusin
bully : bully
maksudnya : maksud
satire : satire
menodai : noda
etis : etis
moslem : moslem
terkesan : kes
wagu

temennya : temennya
versimu : versi
soalnya : soal
versiku : versi
ng : ng
dikaitkan : kait
emosi : emosi
belom : bom
kayanya : kaya
malu : malu
alasan : alas
komenan : komenan
mereka : mereka
alibinya : alibi
mbahmu : mbah
christopher : christopher
columbus : columbus
kang : kang
tiktod : tiktod
kakak : kakak
imo : imo
ajasih : ajasih
lg : lg
asuuu : asuuu
tiktoo : tiktoo
kontol : kontol
gambaran : gambar
nihilisme : nihilisme
nilai : nilai
tusk : tusk
horror : horror
komedinya : komedi
asia : asia
eneg : eneg
nontonnya : nontonnya
dinasehatin : dinasehatin
off : off
yaelah : yaelah
asik : asik
santai : santai
bos : bos
balapan : balap
bejat : bejat
setan : setan
elujie : elujie
martabak : martabak
paittt : paittt
apanya : apa
semenjak : semenjak
lupa : lupa
peng : peng
buk : buk
ngajak : ngajak
dibilikan : bilik
yatim piatu : yatim piatu
kuajak : ajak
kubelii : beli
tegor : tegor
gangerti : gangerti
ato : ato
alias : alias
menghindar : hindar
haram : haram
canda : canda
air : air
kaw

naini : naini
darknya : darknya
waw : waw
tersungging : sungging
kejauhan : jauh
lembek : lembek
otak : otak
gwe : gwe
motret : motret
mahir : mahir
yha : yha
iyhabapak : iyhabapak
maen : maen
diskusi : diskusi
bermana : mana
dian : dian
fuckin : fuckin
jhon : jhon
merindu : rindu
mendengarkan : dengar
ever : ever
mantap : mantap
digituin : digituin
timbang : timbang
dijahit : jahit
lumpuh : lumpuh
yo : yo
ngp : ngp
mikirnya : mikirnya
jujur : jujur
pikir : pikir
dingin : dingin
madam : madam
selin : lin
sebar : sebar
bet : bet
isilop : isilop
humoris : humoris
dikasi : kasi
berkeliaran : liar
empatinya : empati
abnormal : abnormal
berlindung : lindung
anyone : anyone
salty : salty
theyll : theyll
tf : tf
adoh : adoh
adminnya : adminnya
minder : minder
bgtu : bgtu
milopppp : milopppp
kriminalku : kriminal
frekuensi : frekuensi
butuhin : butuhin
dengerin : dengerin
binik : bik
jer : jer
dala : dala
rakyat : rakyat
pakar : pakar
peneliti : teliti
menganalisis : analis
kahh : kahh
satanic

darkamppolitical : darkamppolitical
apapun : apa
tastenya : tastenya
katrok : katrok
iqnya : iqnya
jongkok : jongkok
mendingan : mending
mahamin : mahamin
ilang : ilang
selam : selam
jadikan : jadi
pembenaran : benar
ketemu : ketemu
remaja : remaja
heyyo : heyyo
wazzap : wazzap
nigga : nigga
topik : topik
nyaman : nyaman
memperdulikan : duli
konsi : konsi
malfoy : malfoy
gapaham : gapaham
mega : mega
perdebatan : debat
bego : bego
soan : soan
heh : heh
hoh : hoh
tidak pernah : tidak pernah
kaku : kaku
ma : ma
broo : broo
merespon : merespon
sbg : sbg
ditaro : ditaro
partisipasinya : partisipasi
pandangan : pandang
komennya : komennya
salaa : salaa
omongin : omongin
inget : inget
retweet : retweet
hi : hi
babe : babe
point : point
te : te
mampir : mampir
gag : gag
kunjungi : kunjung
contoh : contoh
jogja : jogja
aamiin : aamiin
prioritasin : prioritasin
ditambah : tambah
kontakan : kontak
dikasuskan : kasus
penistaan : nista
hakikatnya : hakikat
jawa : jawa
woles : woles
nuranimu : nura

...

literal : literal
irl : irl
dipakein : dipakein
mingguan : minggu
sefrontal : frontal
sejelas : jelas
sarkasnyinyir : sarkasnyinyir
jsshwodhsidh : jsshwodhsidh
jumpol : jumpol
pembandingnya : banding
theologia : theologia
bills : bills
sarkasnyindir : sarkasnyindir
laura : laura
babying : babying
gilakkkk : gilakkkk
seringkali : seringkali
dijaw : dijaw
grontol : grontol
pemecatan : pecat
hdhhh : hdhhh
thankss : thankss
char : char
hydro : hydro
def : def
ueueueue : ueueueue
yairi : yairi
convince : convince
childfree : childfree
sebenr : sebenr
php : php
kasusmu : kasus
devita : devita
np : np
ena : ena
akunku : akun
bersihcuma : bersihcuma
sieta : sieta
lndrone : lndrone
tc : tc
qw : qw
koh : koh
doong : doong
nala : nala
berhati : hati
tubirnya : tubir
gre : gre
huahahaha : huahahaha
sayu : sayu
thoma : thoma
contentnya : contentnya
harap : harap
kakliat : kakliat
lokasinyaa : lokasinyaa
graciella : graciella
negasin : negasin
deklarator : deklarator
deklarasi : deklarasi
pidato : p

slide : slide
door : door
cengel : cengel
pegel : gel
elijahahr : elijahahr
money : money
penyanyi : nyanyi
caf : caf
gaperlu : gaperlu
miku : mi
jawabanku : jawab
menohok : tohok
mole : mole
dijawabnya : jawab
mts : mts
rombongannya : rombong
bodyguard : bodyguard
wangi : wangi
bodoamatan : bodoamatan
ekwkwkw : ekwkwkw
jiahahaha : jiahahaha
rekapan : rekap
absen : absen
dikumpul : kumpul
pemarah : marah
wkkwkwkwkwk : wkkwkwkwkwk
hayooo : hayooo
pemiliknya : milik
berkoarkoar : berkoarkoar
persib : persib
didalem : dalem
menyebabkan : sebab
multita : multita
pilihanr : pilihanr
farhat : farhat
abbas : abbas
ngadmin : ngadmin
emotikon : emotikon
wkwkwkwi : wkwkwkwi
gembokan : gembok
sarkaslo : sarkaslo
mengelak : elak
menantang : tantang
ketiduran : tidur
lnda : lnda
matmin : matmin
blio : blio
hwkwhdksgd : hwkwhdksgd
menolak : tolak
mentahmentah : mentahmentah
jabang : jabang
nderrr : nderrr
parkirnya : parkir
sebodoh : bodoh
dokday : dokday
jeva : jeva
kataku : kata
rrrr : rrrr
kekiri

matamata : matamata
nanar : nanar
berbinar : binar
wabah : wabah
menyebar : sebar
qaqa : qaqa
berjamaah : jamaah
ecommurz : ecommurz
gamenya : gamenya
klockren : klockren
mengutipmert : mengutipmert
mengonggongiku : mengonggongiku
twitku : twitku
kemarahan : marah
membatas : batas
perb : perb
kekecewaannya : kecewa
kesibukan : sibuk
alain : alain
botton : botton
nyiny : nyiny
sungguhan : sungguh
ostrali : ostrali
pai : pai
layani : layan
semapan : map
mapannya : mapan
milyar : milyar
termas : mas
menyarankan : saran
ngaj : ngaj
kmunngsenj : kmunngsenj
maah : maah
opposition : opposition
ks : ks
omdia : omdia
vax : vax
msggg : msggg
berintegritas : integritas
peraturan : atur
vans : vans
navy : navy
skhi : skhi
distir : distir
mahilayei : mahilayei
utamanya : utama
koch : koch
jastok : jastok
omkalu : omkalu
dirubung : rubung
laler : laler
peesi : peesi
densilah : densilah
gcp : gcp
aws : aws
fintech : fintech
setingan : ting
musl : musl
piano : piano
tranpose : tranpose
modulasi : modu

Pandas Apply:   0%|          | 0/21636 [00:00<?, ?it/s]

0                                                  [follo]
1                        [desain, rumah, cinta, darkjokes]
2                                [onani, mending, korupsi]
3        [sayang, jgn, marah, ayo, natal, lucu, istighf...
4        [bahas, politik, perintah, kelas, epic, gak, b...
                               ...                        
21631    [jajan, mulu, ngerokok, mulu, nabung, gimana, ...
21632    [errorist, of, seasons, buruh, korban, phk, co...
21633                   [satir, buat, lemah, paham, cinta]
21634    [gus, nur, kritis, said, aqil, yaqut, penjara,...
21635    [wn, you, should, learn, english, if, you, can...
Name: tweet_tokens_stemmed, Length: 21636, dtype: object


In [6]:
data.to_csv("hasilpreproces.csv")